In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

In [ ]:
def loadLDPC(filename):
    A = sio.loadmat(filename)
    G = A['G']
    H = A['H']
    return G, H

In [ ]:
def apply_channel_noise(y, epsilon):
    ## TODO, complement each bit with probability epsilon
    return np.array([i if np.random.rand() > epsilon else not i for i in y])

In [ ]:
def encode_message(x, G):
    ## TODO, implement Gx % 2 :-))
    new_message = np.dot(G,x)
    return new_message % 2

---

In [ ]:
class FactorTypeOne():
    def __init__(self, y_til, epsilon):
        self.y_til = y_til
        self.epsilon = epsilon
    
    def calculate_value(self, y):
        return self.epsilon if y != self.y_til else 1 - self.epsilon

In [ ]:
class FactorTypeTwo():
    def __init__(self, scope):
        # consider a factor: \phi(1, 4, 6), so in this case scope = [1,4,6]
        self.scope = np.array(scope)
    
    def calculate_value(self, scope_assignment):
        # if sum(scope_assignment) is even, the value = 1, O.W 0
        return 1 if sum(scope_assignment) % 2 == 0 else 0
        

In [ ]:
class FactorGraph():
    
    def __init__(self, H, epsilon, y_tilde):
        self.factors_type1 = [] # list of FactorTypeOne
        self.factors_type2 = [] # list of FactorTypeTwo
        self.var_to_factor = {} # map --> (var, [factors related to this var])
        self.factor_to_var = {} # map --> (factor, [vars related to this factor])
        self.messagesVarToFactor = {}
        self.messagesFactorToVar = {}
        
        for i,b in enumerate(y_tilde):
            self.factors_type1.append(FactorTypeOne(y_tilde[i], epsilon))
            self.var_to_factor[i] = [(1, len(self.factors_type1) - 1), ] # 1 means that the factor is from the first type
            self.factor_to_var[(1, len(self.factors_type1) - 1)] = [i, ] # 1 means that the factor is from the first type
        
        for row in H:
            scope = [var for var in range(len(y_tilde)) if row[var] == 1]
            self.factors_type2.append(FactorTypeTwo(scope))
            
            for i in scope:
                self.var_to_factor[i].append((2, len(self.factors_type2) - 1)) # 2 means that the factor is from the 2nd type
                
            self.factor_to_var[(2, len(self.factors_type2) - 1)] = scope       # 2 means that the factor is from the 2nd type
        
        
    ############################################################################################################       
        
        
    def assignment_probability(self, assignment):
        prob = 1
        
        # For unary Factors:
        for i, b in enumerate(assignment):
            prob_this_bit = self.factors_type1[i].calculate_value(b)  #  TODO: implement the easy single line to compute the value of this factor
            prob *= prob_this_bit
        
        # Second Type
        for f2 in self.factors_type2:
            scope = assignment[f2.scope] #  TODO: compute the scope assignment of this factor, due to the given assignment
            prob *= f2.calculate_value(scope)
            
        return prob
    
    
    ############################################################################################################       
    
    def normalize(self, arr):
        return arr / sum(arr) if sum(arr) != 0 else arr
    
    
    def LoopyBP(self, n_iteration):
        
        for ite in range(n_iteration):

            prevMessagesVarToFactor = {}
            prevMessagesFactorToVar = {}
            
            for i, fcts in enumerate(self.var_to_factor):
                factors = self.var_to_factor[fcts]
                for s in factors:
                    if (i,s) not in self.messagesVarToFactor:
                        self.messagesVarToFactor[(i, s)] = np.array([0.5, 0.5])
                    prevMessagesVarToFactor[(i, s)] = self.messagesVarToFactor[(i, s)]
                    
            for s, vrbs in enumerate(self.factor_to_var):
                variables = self.factor_to_var[vrbs]
                for i in variables:
                    if (vrbs, i) not in self.messagesFactorToVar:
                        self.messagesFactorToVar[(vrbs, i)] = np.array([0.5, 0.5])
                    prevMessagesFactorToVar[(vrbs, i)] = self.messagesFactorToVar[(vrbs, i)]
            
            # Update the message var -> factor
            for i, fcts in enumerate(self.var_to_factor):
                factors = self.var_to_factor[fcts]
                for fin in factors:
                    msg = np.array([1.0, 1.0])
                    for fot in factors:
                        if fin != fot:
                            msg[0] *= prevMessagesFactorToVar[(fot, fcts)][0]
                            msg[1] *= prevMessagesFactorToVar[(fot, fcts)][1]
                        self.messagesVarToFactor[(fot, i)] = msg / msg.sum()
                        
            # Update the message factor -> var
            for vr in self.factor_to_var:
                variables = self.factor_to_var[vr]
                if vr[0] == 1:
                    fac2var = np.array([self.factors_type1[vr[1]].calculate_value(i) for i in range(2)])
                    self.messagesFactorToVar[(vr, variables[0])] = fac2var/sum(fac2var)
                else:
                    for s, vrbs in enumerate(variables):
                        marg = np.array([0.0, 0.0])
                        for i in range(2 ** len(variables)):

                            changer = [int(digit) for digit in bin(i)[2:]]
                            for _ in range(len(variables) - len(changer)):
                                changer.insert(0, 0)
                            changer = np.array(changer)
                            
                            var_sum = self.factors_type2[vr[1]].calculate_value(changer)
                            for j, k in enumerate(variables):
                                if k != vrbs:
                                    changer = [int(digit) for digit in bin(i)[2:]]
                                    for _ in range(len(variables) - len(changer)):
                                        changer.insert(0, 0)
                                    changer = np.array(changer)
                                    var_sum *= prevMessagesVarToFactor[(k, vr)][changer[j]]
                            changer = [int(digit) for digit in bin(i)[2:]]
                            for _ in range(len(variables) - len(changer)):
                                changer.insert(0, 0)
                            changer = np.array(changer)
                            marg[changer[s]] += var_sum

                        self.messagesFactorToVar[(vr, vrbs)] = marg/sum(marg)
                
            # Warning: Don't forget to normalize the message at each time.
            
            
            if ite % 10 == 0 and ite > 0:
                print("Finished Loopy Iteration %s" % ite)
    
    
    ############################################################################################################       
    
    
    def estimate_marginal_probability(self, var):
        '''
        This method assumes LoopyBP has been run
        '''
        res = np.array([1.0, 1.0])
        for factor in self.var_to_factor[var]:
            for i in range(2):
                res[i] *= np.array(self.messagesFactorToVar[(factor, var)])[i]
        return res / sum(res)
        
        
    ############################################################################################################       
    
    
    def get_marginal_MAP(self):
        output = np.zeros(256)
        for i, var in enumerate(range(256)):
            output[i] = np.argmax(self.estimate_marginal_probability(i))
        return output
    
        
    

---

In [ ]:
y_tilde = np.array([[1, 1, 1, 1, 1, 1]]).reshape(6, 1)
H = np.array([
        [0, 1, 1, 0, 1, 0],
        [1, 0, 1, 0, 1, 1],
        [0, 1, 0, 1, 1, 0]])
epsilon = 0.05

Graph = FactorGraph(H, epsilon, y_tilde)

ytest1 = np.array([0, 1, 1, 0, 1, 0])
ytest2 = np.array([1, 0, 1, 1, 0, 1])
ytest3 = np.array([1, 0, 1, 1, 1, 1])

print(Graph.assignment_probability(ytest1))
print(Graph.assignment_probability(ytest2))
print(Graph.assignment_probability(ytest3))


0.0
0.0
0.038689046874999994


#### do not apply, they also have a greater number of zeros than the third relation H As we expected, the probability of the first two became zero because
#### The third relation also has a maximum of 1, it differs from the noise message in a device, so it is logical

---

In [ ]:
G, H = loadLDPC('GH.mat')

epsilon = 0.05
N = G.shape[1]
x = np.ones((N, 1), dtype='int32')
y = encode_message(x, G)
yTilde = apply_channel_noise(y, epsilon)

G = FactorGraph(H, epsilon, yTilde)

G.LoopyBP(50)

best_estimation = G.get_marginal_MAP()
print(best_estimation)

D:\software\installs\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


Finished Loopy Iteration 10
Finished Loopy Iteration 20
Finished Loopy Iteration 30
Finished Loopy Iteration 40
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1.
 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1.]


#### The incoming message was all the same and the outgoing message was all the same, so the incoming and outgoing messages were the same
#### is also true H on the other hand the output message in relation